In [267]:
import pandas as pd
from datetime import datetime

In [268]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/bps_coaching_log-main')
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/bps_coaching_log-main


In [269]:
def return_multiple_answers(row):
    return [i for i in row.values if isinstance(i, str)]
def return_single_answer(row):
    strings = return_multiple_answers(row)
    return strings[0] if strings else None

In [270]:
def find_center(df, cols = ["Q2", "Q3", "Q4", "Q6", "Q7", "Q8", "Q9", "Q107"]):
    return df[cols].apply(lambda row: return_single_answer(row), axis=1)

In [271]:
def find_responses_by_question(df, Q):
    cols = [i for i in df.keys() if i.startswith(Q)]
    return df[cols].apply(lambda row: return_multiple_answers(row), axis=1) 

In [272]:
def select_by_date(df, start, end):
    mask = (df['date'] >= start) & (df['date'] <= end)
    return df[mask]

In [273]:
#add the path to the qualtrics survey here
csv_name = "./Boston-UPK+Coaching+Log_April+24+2022_15.41.csv"
survey_results = pd.read_csv(csv_name)

In [274]:
#delete first rows of survey which contain addnl info about the Qs
survey_results2 = survey_results[3:]

#deleting where date is none
survey_results2 = survey_results2.dropna(subset=["Q18#1_1"])

#formatting the date because now it's 3 separate columns
survey_results2["date"] = survey_results2.apply(lambda row: datetime.strptime(str(row["Q18#1_1"]) + str(row["Q18#2_1"]) + str(row["Q18#3_1"]), '%B%d%Y'), axis=1)
survey_results2["month_year"] = survey_results2["date"].apply(lambda x: x.strftime('%m-%Y'))

In [275]:
# Modify dates here if needed, please keep same format!

start = "2021-09-01"
end = "2022-04-29"

start_date = datetime.strptime(start, "%Y-%m-%d")
end_date = datetime.strptime(end, "%Y-%m-%d")

survey_results2 = select_by_date(survey_results2, start_date, end_date)

In [276]:
#creating a new df with only the data we need. this is necessary bc each question in the original csv has Q6_1, Q6_2... 
#etc that are all corresponding to an mcq answer to each Q. ,Only one such column is picked, so we compress all those 
#columns into one in this df, just representing the selected answer in the column. Un dummying if you will. 

df = pd.DataFrame()
df["center"] = find_center(survey_results2)
df["coaching"] = find_responses_by_question(survey_results2, "Q23")
df["curriculum_observed"] = find_responses_by_question(survey_results2, "Q51")
df["focus"]  = find_responses_by_question(survey_results2, "Q53")
df["bb"]  = find_responses_by_question(survey_results2, "Q54")

df["month_year"] = survey_results2["date"].apply(lambda x: x.strftime('%m-%Y'))

In [277]:
def count(df, q_cols = ['coaching', 'curriculum_observed', 'focus', 'bb']):
    centers = set(df['center'])
    centers.remove(None)
    counts_by_center_by_date = {center :{} for center in centers}
    counts_by_center_by_date_by_question = {question:{center :{} for center in centers} for question in q_cols}
    for i, row in df.iterrows():
        center = row['center']
        if center == None:
            continue
        date = row['month_year']
        counts_by_center_by_date[center].setdefault(date, 0)
        counts_by_center_by_date[center][date] += 1
        for question in q_cols:
            responses = row[question]
            for j in responses:
                count = counts_by_center_by_date_by_question[question][center].setdefault(date, {})
                j_type_counts = count.setdefault(j, 0)
                counts_by_center_by_date_by_question[question][center][date][j] += 1 
    return counts_by_center_by_date, counts_by_center_by_date_by_question

In [278]:
def create_session_df_from_dict(dictionary, filename) :   
    dff = pd.DataFrame()
    school = []
    date = []
    sessions_count = []

    for k, v in dictionary.items():
        for i, j in v.items():
            school.append(k)
            date.append(i)
            sessions_count.append(j)       

    dff["school"] = school
    dff["date"] = date
    dff["session_count"] = sessions_count
    dff.to_csv(filename, index = False)
    print(dff.shape)

In [279]:
session_df = create_session_df_from_dict(count(df)[0], "./session.csv")

(94, 3)


In [280]:
def create_other_dfs_from_dict(dictionary, filename):    
    for csv, school_dicts in dictionary.items():
        df = pd.DataFrame()
        school = []
        date = []
        coaching = []
        coaching_counts = []
        for k, v in school_dicts.items():
            for i, j in v.items():
                for coaching_type, coaching_count in j.items():
                    school.append(k)
                    date.append(i)   
                    coaching.append(coaching_type)
                    coaching_counts.append(coaching_count)
        df["school"] = school
        df["date"] = pd.to_datetime(date)
        if csv != 'coaching': df["type"] = [i.split(',') for i in coaching]
        else: df["type"] = coaching
        df["count"] = coaching_counts
        df = df.explode("type",ignore_index=True)
        df.to_csv("./" + csv + filename, index = False)
        print(df.shape)
        print(csv, "done")

In [281]:
create_other_dfs_from_dict(count(df)[1], ".csv")

(124, 4)
coaching done
(206, 4)
curriculum_observed done
(143, 4)
focus done
(88, 4)
bb done


## Qualitative Responses:

In [282]:
def write_out_qualitative_responses(survey_results, start_date, end_date):
    df = select_by_date(survey_results, start_date, end_date)
    #switch qs here if needed:
    q70 = "Briefly describe challenges in the classroom and/or your interaction with the instructional team in this session:"
    q71 = "Briefly describe strengths in the classroom and/or your interaction with the instructional team in this session:"
    cols = [i  for i in survey_results.keys() if i in ["Q70", "Q71"]]
    
    Q70 = pd.DataFrame()
    Q70[q70] =  return_multiple_answers(df["Q70"])
    
    Q71 = pd.DataFrame()
    Q71[q71] =  return_multiple_answers(df["Q71"])
    
    Q70.to_csv("challenges_"  + start + "to" + end + ".csv" , index=False)
    Q71.to_csv("strengths_"  + start + "to" + end + ".csv", index=False)

    print("Qualitative Responses written out!")


In [283]:
write_out_qualitative_responses(survey_results2, start_date, end_date)

Qualitative Responses written out!
